# Multiple Linear Regression
 
## Objectives

After completing this lab, you will be able to:

* Use scikit-learn to implement multiple linear regression
* Create, train, and test a multiple linear regression model on real data


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Importing Data Sets</a>
2. <a href="#item32">Data Wrangling</a>
2. <a href="#item33">Exploratory Data Analysis (EDA)</a> 
4. <a href="#item34">Build The Model</a>
5. <a href="#item35">Model Evaluationl</a>  

</font>
</div>

## Importing Data Sets and Libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [4]:
# Load Data
def load_data(data_file= "../data/FuelConsumptionCo2.csv"):
    data_frame = pd.read_csv(data_file)
    return data_frame

df = load_data()

In [5]:
# Check column infos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067 entries, 0 to 1066
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   MODELYEAR                 1067 non-null   int64  
 1   MAKE                      1067 non-null   object 
 2   MODEL                     1067 non-null   object 
 3   VEHICLECLASS              1067 non-null   object 
 4   ENGINESIZE                1067 non-null   float64
 5   CYLINDERS                 1067 non-null   int64  
 6   TRANSMISSION              1067 non-null   object 
 7   FUELTYPE                  1067 non-null   object 
 8   FUELCONSUMPTION_CITY      1067 non-null   float64
 9   FUELCONSUMPTION_HWY       1067 non-null   float64
 10  FUELCONSUMPTION_COMB      1067 non-null   float64
 11  FUELCONSUMPTION_COMB_MPG  1067 non-null   int64  
 12  CO2EMISSIONS              1067 non-null   int64  
dtypes: float64(4), int64(4), object(5)
memory usage: 108.5+ KB


In [6]:
# verify successful load with some randomly selected records
#df.sample(5)
df.head()

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244


## Data Wrangling

In [6]:
# Count missing values in each column
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]  # only columns with missing values
print(missing_values.sort_values(ascending=False))

zero_counts = (df == 0).sum()
zero_counts = zero_counts[zero_counts > 0]  # only columns with missing values
print("-----------------------------")
print("zero_counts: \n", zero_counts)

Series([], dtype: int64)
-----------------------------
zero_counts: 
 Series([], dtype: int64)


**Comment**: \
The data is clean.

In [7]:
# Drop categoricals and any unseless columns
cdf = df.drop(['MODELYEAR', 'MAKE', 'MODEL', 'VEHICLECLASS', 'TRANSMISSION', 'FUELTYPE'],axis=1)
# Randomly selects 9 rows from the DataFrame
cdf.sample(9)

,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
101,3.0,6,11.9,8.0,10.1,28,232
523,1.6,4,9.1,6.7,8.0,35,184
785,1.6,4,8.6,6.8,7.8,36,179
613,2.0,4,11.7,7.6,9.9,29,228
556,5.0,8,15.8,10.2,13.3,21,306
266,5.3,8,21.5,14.6,18.4,15,294
73,6.0,12,20.0,12.2,16.5,17,380
309,3.6,6,13.2,8.7,11.2,25,258
506,5.0,8,15.7,10.2,13.2,21,304


**Comment**  
- Non numerical variables are not included in the description. We analyze these features if required to improve the accuracy of your model.  
-  MODELYEAR is the same for all cars, so you can drop these variables for this modeling illustration.

<h2 id="item33">Exploratory Data Analysis (EDA)</h2>

In [8]:
df.describe()

,MODELYEAR,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
count,1067.0,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000,1067.000000
mean,2014.0,3.346298,5.794752,13.296532,9.474602,11.580881,26.441425,256.228679
std,0.0,1.415895,1.797447,4.101253,2.794510,3.485595,7.468702,63.372304
min,2014.0,1.000000,3.000000,4.600000,4.900000,4.700000,11.000000,108.000000
25%,2014.0,2.000000,4.000000,10.250000,7.500000,9.000000,21.000000,207.000000
50%,2014.0,3.400000,6.000000,12.600000,8.800000,10.900000,26.000000,251.000000
75%,2014.0,4.300000,8.000000,15.550000,10.850000,13.350000,31.000000,294.000000
max,2014.0,8.400000,12.000000,30.200000,20.500000,25.800000,60.000000,488.000000


In [9]:
##  Check the size of the data points
df.shape

(1067, 13)

In [10]:
# The correlation matrix displays the pairwise correlations between all features
cdf.corr()

,ENGINESIZE,CYLINDERS,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
ENGINESIZE,1.000000,0.934011,0.832225,0.778746,0.819482,-0.808554,0.874154
CYLINDERS,0.934011,1.000000,0.796473,0.724594,0.776788,-0.770430,0.849685
FUELCONSUMPTION_CITY,0.832225,0.796473,1.000000,0.965718,0.995542,-0.935613,0.898039
FUELCONSUMPTION_HWY,0.778746,0.724594,0.965718,1.000000,0.985804,-0.893809,0.861748
FUELCONSUMPTION_COMB,0.819482,0.776788,0.995542,0.985804,1.000000,-0.927965,0.892129
FUELCONSUMPTION_COMB_MPG,-0.808554,-0.770430,-0.935613,-0.893809,-0.927965,1.000000,-0.906394
CO2EMISSIONS,0.874154,0.849685,0.898039,0.861748,0.892129,-0.906394,1.000000


**Comment**  
1. Look at the bottom row, which shows the correlation between each variable and the target, 'CO2EMISSIONS'. Each of these shows a fairly high level of correlation, each exceeding 85% in magnitude.

2. Examine the correlations of the distinct pairs. 'ENGINESIZE' and 'CYLINDERS' are highly correlated, but 'ENGINESIZE' is more correlated with the target, so we can drop **'CYLINDERS'**. 

3. Similarly, each of the four fuel economy variables is highly correlated with each other. Since FUELCONSUMPTION_COMB_MPG is the most correlated with the target, you can drop the others: **'FUELCONSUMPTION_CITY,' 'FUELCONSUMPTION_HWY,' 'FUELCONSUMPTION_COMB.'** 

4. The  **FUELCONSUMPTION_COMB** and **FUELCONSUMPTION_COMB_MPG** are not perfectly correlated. They should be, though, because they measure the same property in different units. In practice, you would investigate why this is the case. You might find out that some or all of the data is not useable as is.

5. Remove redundant parameters from regression
To help with selecting predictive features that are not redundant, consider the following scatter matrix, which shows the scatter plots for each pair of input features. The diagonal of the matrix shows each feature's histogram.


In [ ]:
cdf = cdf.drop(['CYLINDERS', 'FUELCONSUMPTION_CITY', 'FUELCONSUMPTION_HWY','FUELCONSUMPTION_COMB',],axis=1)
cdf.sample(9)

In [ ]:
axes = pd.plotting.scatter_matrix(cdf, alpha=0.2)
# rotate axis labels so we can read them
for ax in axes.flatten():
    ax.xaxis.label.set_rotation(90)
    ax.yaxis.label.set_rotation(0)
    ax.yaxis.label.set_ha('right')

plt.tight_layout()
plt.gcf().subplots_adjust(wspace=0, hspace=0)
plt.show()
    

**Comment** 

The relationship between **'FUELCONSUMPTION_COMB_MPG'**  and **'CO2EMISSIONS'**  is non-linear. 
 - This suggests exploring the categorical variables to see if they are able to explain these differences.
 - Regarding the non-linearity, you will handle this in the next lab. For now, let's just consider through modeling whether fuel economy explains some of the variances in the target as is.


<h2 id="item34">Build The Model</h2>

In [ ]:
### 1. Extract the input feature and labels from the dataset
X = cdf.iloc[:,[0,1]].to_numpy() ## represents ENGINESIZE and FUELCONSUMPTION_COMB_MPG	
y = cdf.iloc[:,[2]].to_numpy() ## represents CO2EMISSIONS

# 2. standardize the input features so the model doesn't inadvertently favor any feature due to its magnitude
from sklearn import preprocessing
std_scaler = preprocessing.StandardScaler()
X_norm = std_scaler.fit_transform(X)

pd.DataFrame(X_norm).describe().round(2)

**Comment**:
As you can see, a standardized variable has zero mean and a standard deviation of one.

In [ ]:
# 3.Create train and test datasets
### Randomly split your data into train and test sets, using 80% of the dataset for training and reserving the remaining 20% for testing.
X_train, X_test, y_train, y_test = train_test_split(X_norm,y,test_size=0.2,random_state=42)
## Print the data dimensions
type(X_train), np.shape(X_train), np.shape(X_train)

**Comment**:
The outputs are one-dimensional NumPy arrays or vectors.

In [ ]:
from sklearn import linear_model

# 4. create a model object
regressor_model = linear_model.LinearRegression()

# 5. train the model in the training data
#X_train is a 2-D array as sklearn models expects, there is no need to shape it
regressor_model.fit(X_train, y_train)

# Print the coefficients
coef_ =  regressor_model.coef_
intercept_ = regressor_model.intercept_
# Print the coefficients
print ('Coefficients: ',coef_)
print ('Intercept: ',intercept_)

**Comment**:
- __Coefficients__ and __Intercept__ are the regression parameters determined by the model. 
- They define the best-fit hyperplane to the data. Since there are only two variables, hence two parameters, the hyperplane is a plane. 

## Model Evaluation

In [ ]:
# Evaluation 
# Ensure X1, X2, and y_test have compatible shapes for 3D plotting
print(f"X_test is a {X_test.ndim}-D Matrix of Shape {X_test.shape}")
print(f"y_test is a {y_test.shape[1]}-D Matrix of Shape {y_test.shape}")
# each of the two columns represent one feature X1:ENGINESIZE and	X2: FUELCONSUMPTION_COMB_MPG
x_energy_size_test = X_test[:, 0] if X_test.ndim > 1 else X_test
x_consumption_comb_mpg_test = X_test[:, 1] if X_test.ndim > 1 else np.zeros_like(X1)

# Predict y values using trained regression model to compare with actual y_test
#y_pred = regressor_model.predict(X_test.reshape(-1, 1)) if X_test.ndim == 1 else regressor_model.predict(X_test)
y_pred = regressor_model.predict(X_test)
print("Mean absolute error: %.2f" % mean_absolute_error(y_test, y_pred))
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2-score: %.2f" % r2_score(y_test, y_pred))

# Plotting KDE for Observed vs. Predicted Values
plt.figure(figsize=(8, 5))
#sns.set()
#sns.set_theme()
#sns.set_style("darkgrid")
sns.kdeplot(y_test, label='Actual', color='blue')
sns.kdeplot(y_pred, label='Predicted', fill=True, color='green', alpha=0.4)
plt.xlabel('Target Variable')
plt.ylabel('Density')
plt.title('KDE Plot of Actual vs. Predicted Values')
plt.legend()
plt.show()

In [ ]:
# Get the standard scaler's mean and standard deviation parameters
means_ = std_scaler.mean_
std_devs_ = np.sqrt(std_scaler.var_)

# The least squares parameters can be calculated relative to the original, unstandardized feature space as:
coef_original = coef_ / std_devs_
intercept_original = intercept_ - np.sum((means_ * coef_) / std_devs_)
# Print the coefficients
print ('Coefficients: ', coef_original)
print ('Intercept: ', intercept_original)

__Comment:__
- Transforming the model's parameters back to the original space gives a proper sense of what they mean in terms of your original input features. Without the previous adjustments, the model's outputs would be tied to an abstract, transformed space that doesn’t align with the actual independent variables and the real-world problem you’re solving.
- You would expect that for the limiting case of zero **ENGINESIZE** and zero **FUELCONSUMPTION_COMB_MPG**, the resulting CO2 emissions should also be zero (if no engine then no emission). 
- This is inconsistent with the 'best fit' hyperplane, which has a non-zero __Intercept__ of 329 g/km. this means that the target variable __does not have a very strong linear relationship to the dependent variables__, and/or the data has __outliers__ that are biasing the result.
- Outliers can be handled in preprocessing..

In [ ]:
x_engine_size_train = X_train[:, 0]
x_consumption_comb_mpg_train = X_train[:, 1]
y_emission_pred_0 = coef_[0, 0] * X_train[:, 0] + intercept_[0]
y_emission_pred_1 = coef_[0,1] * X_train[:,1] + intercept_[0]

fig, axes = plt.subplots(1, 2, figsize=(15, 8))

#  Scatter on axis (no assignment!)
axes[0].scatter(
    x_engine_size_train,
    y_train,
    label="Train"
)

#   Plot regression line on same axis
axes[0].plot(
    x_engine_size_train,
    y_emission_pred_0,
    color="red",
    linestyle="-",
    label="Predicted"
)

axes[0].set_ylim(0, None)
axes[0].set_xlabel("Engine size")
axes[0].set_ylabel("Emission")
axes[0].legend()

#  Scatter on axis (no assignment!)
axes[1].scatter(
    x_consumption_comb_mpg_train,
    y_train,
    label="Train"
)

#   Plot regression line on same axis
axes[1].plot(
    x_consumption_comb_mpg_train,
    y_emission_pred_1,
    color="red",
    linestyle="-",
    label="Predicted"
)
axes[1].set_ylim(0, None)
axes[1].set_xlabel("FUELCONSUMPTION_COMB_MPG")
axes[1].set_ylabel("Emission")
axes[1].legend()

plt.show()



In [ ]:
# Create side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Residuals for Engine Size
sns.residplot(
    x=x_engine_size_train,
    y=y_train,
    ax=axes[0]
)
axes[0].set_title("Residuals vs Engine Size")
axes[0].set_xlabel("Engine Size")
axes[0].set_ylabel("Residuals")

# Residuals for Fuel Consumption
sns.residplot(
    x=x_consumption_comb_mpg_train,
    y=y_train,
    ax=axes[1]
)
axes[1].set_title("Residuals vs Fuel Consumption")
axes[1].set_xlabel("FUELCONSUMPTION_COMB_MPG")
axes[1].set_ylabel("Residuals")

plt.tight_layout()
plt.show()


__Comment:__ the solution is incredibly poor because the model is trying to fit a plane to a non-planar surface.


### Exercise 1
Determine and print the parameters for the best-fit linear regression line for CO2 emission with respect to engine size.


In [ ]:
regressor_1 = linear_model.LinearRegression()
# X_train is a 1-D array but sklearn models expect a 2D array as input for the training data, with shape (n_observations, n_features).
# So we need to reshape it. We can let it infer the number of observations using '-1'.
regressor_1.fit(x_engine_size_train.reshape(-1, 1), y_train)
coef_1 =  regressor_1.coef_
intercept_1 = regressor_1.intercept_

print ('Coefficients: ',coef_1)
print ('Intercept: ',intercept_1)


### CO2 emission against ENGINESIZE (training/linear regression)
Produce a scatterplot of CO2 emission against ENGINESIZE and include the best-fit regression line to the training data.  


In [ ]:
# Enter your code here
plt.scatter(x_engine_size_train, y_train)
plt.plot( x_engine_size_train, coef_1[0] * x_engine_size_train + intercept_1, '-r')
plt.xlabel("Engine size")
plt.ylabel("Emission")

Evidently, this simple linear regression model provides a much better fit of CO2 emissions on the training data than the multiple regression model did.


###  CO2 emission against ENGINESIZE  (test/linear regression)
Generate the same scatterplot and best-fit regression line, but now base the result on the test data set. 
Consider how the test result compares to the training result.


In [ ]:
X_test_1 = X_test[:,0]
plt.scatter(X_test_1, y_test,  color='blue')
plt.plot(X_test_1, coef_1[0] * X_test_1 + intercept_1, '-r')
plt.xlabel("Engine size")
plt.ylabel("CO2 Emission")

### CO2 emission against FUELCONSUMPTION_COMB_MPG (training/linear regression)
Repeat the same modeling but use FUELCONSUMPTION_COMB_MPG as the independent variable instead. Display the model coefficients including the intercept.


In [ ]:
X_train_2 = X_train[:,1]
regressor_2 = linear_model.LinearRegression()
regressor_2.fit(X_train_2.reshape(-1, 1), y_train)
coef_2 =  regressor_2.coef_
intercept_2 = regressor_2.intercept_
print ('Coefficients: ',coef_2)
print ('Intercept: ',intercept_2)

Generate a scatter plot showing the results as before on the test data.
Consider  well the model fits, and what you might be able to do to improve it. We'll revisit this later in the course.


In [ ]:

X_test_2 = X_test[:,1]
plt.scatter(X_test_2, y_test , color='blue')
plt.plot(X_test_2, coef_2[0] *X_test_2+ intercept_2, '-r')
plt.xlabel("Fuel Consumption")
plt.ylabel("CO2 Emission")
